In [ ]:
import pandas as pd

Get raw data from pubmed 

In [ ]:
! wget ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/CID-Synonyms.gz && gunzip CID-Synonyms.gz && mv CID-Synonyms CID-Synonyms.tsv
! wget ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/CID-SMILES.gz && gunzip CID-SMILES.gz && mv CID-SMILES CID-SMILES.tsv

Read data 

In [ ]:
CID_SMILES = "./CID-SMILES.tsv"
CID_SYNONYM = "./CID-Synonyms.tsv"
COMPOUNDS = "./compounds.csv"

In [ ]:
cid_smiles = pd.read_csv(CID_SMILES, sep="\t").rename(
    columns={"1": "CID", "CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C": "smiles"}
)
cid_smiles

,CID,smiles
0,2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
1,3,C1=CC(C(C(=C1)C(=O)O)O)O
2,4,CC(CN)O
3,5,C(C(=O)COP(=O)(O)O)N
4,6,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl
...,...,...
122454452,176730691,CC1(CCN(CC1)C2=C(N=CC=C2)N)OC(=O)NC(C)(C)C
122454453,176730692,CC#CC1=CC(=C(C(=C1C(F)(F)F)[C@@H]2CC3=C(CO2)C(...
122454454,176730693,CC[C@H](C)[C@@H](C(=O)C(=O)[C@H](CC1=CC=CC=C1)...
122454455,176730694,CC[C@H](C)[C@@H](C(=O)C(=O)[C@H](CC1=CC=CC=C1)...


In [ ]:
cid_synonyms = pd.read_csv(CID_SYNONYM, sep="\t")
cid_synonyms

,1,acetylcarnitine
0,1,Acetyl-DL-carnitine
1,1,Acetyl-L-carnitine
2,1,Acetyl carnitine
3,1,DL-O-Acetylcarnitine
4,1,DL-Acetylcarnitine
...,...,...
174493328,176730692,5-((S)-7-(3-amino-2-fluoro-5-(prop-1-yn-1-yl)-...
174493329,176730693,EXENATIDE ACETATE
174493330,176730693,141732-76-5
174493331,176730695,(S)-3-((((9H-fluoren-9-yl)methoxy)carbonyl)ami...


Read compounds

In [ ]:
compounds = pd.read_csv(COMPOUNDS)
CAS = compounds["CAS Number"].values

In [ ]:
res = (
    cid_synonyms[cid_synonyms["acetylcarnitine"].isin(CAS)]
    .dropna()
    .reset_index(drop=True)
    .rename(columns={"1": "CID", "acetylcarnitine": "CAS"})
)
res

,CID,CAS
0,2,5080-50-2
1,7,2715-68-6
2,19,303-38-8
3,43,103404-90-6
4,47,1460-34-0
...,...,...
45831,176541418,474-25-9
45832,176542861,13292-46-1
45833,176544248,129-16-8
45834,176546160,3810-74-0


In [ ]:
merged_df = pd.merge(res, cid_smiles, on="CID")
merged_df.to_csv("./CAS_to_smiles.tsv", sep="\t")

merged_df

,CID,CAS,smiles
0,2,5080-50-2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
1,7,2715-68-6,CCN1C=NC2=C(N=CN=C21)N
2,19,303-38-8,C1=CC(=C(C(=C1)O)O)C(=O)O
3,43,103404-90-6,C(CC(=O)O)C(C(=O)O)O
4,47,1460-34-0,CCC(C)C(=O)C(=O)O
...,...,...,...
45831,176541418,474-25-9,C[C@H](CCC(=O)O)[C@H]1CCC2[C@@]1(CC[C@@H]3[C@@...
45832,176542861,13292-46-1,C[C@H]1/C=C(/C=C(/C(=O)NC2=C(C(=C3C(=C2O)C(=C(...
45833,176544248,129-16-8,C1=CC=C2C(=C1)C(=O)OC23C4=CC(=C(C=C4OC5=C=C([C...
45834,176546160,3810-74-0,C[C@@H]1[C@]([C@@H]([C@H](O1)O[C@@H]2[C@H]([C@...


Mege CAS with compound metadata 

In [ ]:
df = pd.merge(compounds, merged_df, left_on="CAS Number", right_on="CAS", how="left")

df

,Ranking,Catalog Number,Score,Compound name,Treatment concentration,CAS Number,CID,CAS,smiles
0,1,HY_107369,0.367344,4-Butylresorcinol,10 μM,18979-61-8,205912.0,18979-61-8,CCCCC1=C(C=C(C=C1)O)O
1,2,HY_119674A,0.366090,Xanthopterin (hydrate),10 μM,5979-01-1,261738.0,5979-01-1,C1=CC=C(C=C1)C(C2=CC=CC=C2)(C3=CC=CC=C3)OCC4C(...
2,2,HY_119674A,0.366090,Xanthopterin (hydrate),10 μM,5979-01-1,271303.0,5979-01-1,CC1(CCCC2(C1CC(=O)C3=C2C(=O)C4C(C3)(O4)C(C)(C)...
3,2,HY_119674A,0.366090,Xanthopterin (hydrate),10 μM,5979-01-1,24870768.0,5979-01-1,C1=NC2=NC(=NC(=O)C2=NC1=O)N
4,2,HY_119674A,0.366090,Xanthopterin (hydrate),10 μM,5979-01-1,78291276.0,5979-01-1,C1=NC2=NC(=NC(=O)C2=NC1=O)N.O
...,...,...,...,...,...,...,...,...,...
50038,12354,HY_13515,-0.277132,Sirtinol,10 μM,410536-97-9,1376646.0,410536-97-9,C[C@H](C1=CC=CC=C1)NC(=O)C2=CC=CC=C2N=CC3=C(C=...
50039,12354,HY_13515,-0.277132,Sirtinol,10 μM,410536-97-9,2827646.0,410536-97-9,CC(C1=CC=CC=C1)NC(=O)C2=CC=CC=C2N=CC3=C(C=CC4=...
50040,12355,HY_B2111,-0.282980,(R)-(+)-Atenolol,10 μM,56715-13-0,2249.0,56715-13-0,CC(C)NCC(COC1=CC=C(C=C1)CC(=O)N)O
50041,12355,HY_B2111,-0.282980,(R)-(+)-Atenolol,10 μM,56715-13-0,175540.0,56715-13-0,CC(C)NC[C@@H](COC1=CC=C(C=C1)CC(=O)N)O


Drop all ambiguous matches

In [ ]:
x = df["Catalog Number"].value_counts() == 1
unique_compounds = x.loc[x].index
unique_compounds

Index(['HY_N2393', 'HY_131159', 'HY_12879', 'HY_19702', 'HY_122641B',
       'HY_W018501', 'HY_18676B', 'HY_19762', 'HY_19903', 'HY_19910',
       ...
       'HY_W039454', 'HY_13646A', 'HY_126225', 'HY_100736', 'HY_111325',
       'HY_79511', 'HY_P2526A', 'HY_B1387', 'HY_12297', 'HY_107369'],
      dtype='object', name='Catalog Number', length=2519)

In [ ]:
df.loc[df["Catalog Number"].isin(unique_compounds)].to_csv(
    "unique_compounds.tsv", sep="\t"
)